In [1]:
import os
os.chdir("../")
%pwd

'c:\\Code Playground\\Programs_github\\Chicken-Disease-Classification-project'

In [6]:
# Update Config.yaml
# update Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig: 
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
# Update Configurations manager
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager: 
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH): 
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig: 
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir), 
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
        )

        return training_config

In [8]:
# Update Components
import time

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
# Create new component

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class Training: 
    def __init__(self, config: TrainingConfig): 
        self.config = config
    
    def get_base_model(self): 
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self): 
        
        datagenerator_kawrgs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kawrgs
        )

        self.valid_generator = valid_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False, 
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation: 
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2, 
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kawrgs
            )
        else: 
            train_datagen = valid_datagen

        self.train_generator = train_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model): 
        model.save(path)

    def train(self, callback_list: list): 
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.history = self.model.fit(
            self.train_generator, 
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps = self.validation_steps,
            callbacks = callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model = self.model
        )
        


In [12]:
# Create pipeline
try: 
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list = callback_list)

except Exception as e:
    raise e

[2023-07-30 16:21:49,781: INFO: common] yaml file: config\config.yaml loaded successfully
[2023-07-30 16:21:49,784: INFO: common] yaml file: params.yaml loaded successfully
[2023-07-30 16:21:49,786: INFO: common] directory created at: artifacts
[2023-07-30 16:21:49,788: INFO: common] directory created at: artifacts\prepare_callbacks\checkpoint_dir
[2023-07-30 16:21:49,790: INFO: common] directory created at: artifacts\prepare_callbacks\tensorboard_log_dir
[2023-07-30 16:21:49,792: INFO: common] directory created at: artifacts\training
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/10
19/19 [==============================] - 45s 2s/step - loss: 18.0343 - accuracy: 0.4459 - val_loss: 3.2110 - val_accuracy: 0.5469
Epoch 2/10


c:\Users\rahul\miniconda3\envs\chicken\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


19/19 [==============================] - 45s 2s/step - loss: 3.8158 - accuracy: 0.7669 - val_loss: 1.1446 - val_accuracy: 0.8438
Epoch 3/10
19/19 [==============================] - 46s 2s/step - loss: 2.4183 - accuracy: 0.8378 - val_loss: 0.6527 - val_accuracy: 0.8750
Epoch 4/10
19/19 [==============================] - 46s 2s/step - loss: 2.9724 - accuracy: 0.7736 - val_loss: 0.6678 - val_accuracy: 0.9062
Epoch 5/10
19/19 [==============================] - 46s 2s/step - loss: 1.2500 - accuracy: 0.8480 - val_loss: 0.8082 - val_accuracy: 0.9062
Epoch 6/10
19/19 [==============================] - 46s 2s/step - loss: 0.9003 - accuracy: 0.9020 - val_loss: 4.7243 - val_accuracy: 0.6094
Epoch 7/10
19/19 [==============================] - 47s 2s/step - loss: 3.0226 - accuracy: 0.7770 - val_loss: 1.3264 - val_accuracy: 0.7969
Epoch 8/10
19/19 [==============================] - 46s 3s/step - loss: 0.5347 - accuracy: 0.9189 - val_loss: 3.1334 - val_accuracy: 0.7344
Epoch 9/10
19/19 [=============